<a href="https://colab.research.google.com/github/AliAch04/build-llm-from-scratch/blob/main/LLMScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Short Story as text sample Into python

**Step 1 : Create Tokens**

In [2]:
with open("External/the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()

print('Number of characters : ', len(text))
# print first 100 characters
print(text[:99])

Number of characters :  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- Tokenize the whole characters (20479) into individual words and special characters Then turn into embeddings

- Split text into list of tokens based on white text or special characters ...

In [7]:
import re # Regular expression

text_test = "Hello world. this, is a test."
result = re.split(r'(\S)', text_test)

print(result)

['', 'H', '', 'e', '', 'l', '', 'l', '', 'o', ' ', 'w', '', 'o', '', 'r', '', 'l', '', 'd', '', '.', ' ', 't', '', 'h', '', 'i', '', 's', '', ',', ' ', 'i', '', 's', ' ', 'a', ' ', 't', '', 'e', '', 's', '', 't', '', '.', '']
